In [ ]:
from src.model.model_ast import ASTModelWrapper
import librosa
import torch
import torchaudio
from pytorch_lightning.loggers import TensorBoardLogger
from transformers import ASTConfig, ASTFeatureExtractor, ASTForAudioClassification, ASTModel
from transformers.modeling_outputs import SequenceClassifierOutput
import src.config.config_defaults as config_defaults


model_name =  config_defaults.DEFAULT_AST_PRETRAINED_TAG
num_labels = config_defaults.DEFAULT_NUM_LABELS

config = ASTConfig.from_pretrained(
	pretrained_model_name_or_path=model_name,
)
 
model = ASTModel.from_pretrained(
	model_name,
	config=config,
	ignore_mismatched_sizes=True,
)



In [ ]:
from src.data.dataset_irmas import IRMASDatasetTrain
import torch.utils.data

from src.features.audio_transform import AudioTransformAST

audio_transform =  AudioTransformAST(
    sampling_rate=config_defaults.DEFAULT_SAMPLING_RATE,
    pretrained_tag=config_defaults.DEFAULT_AST_PRETRAINED_TAG,
    augmentation_enums=[],
)

train_dataset = IRMASDatasetTrain(audio_transform=audio_transform)
training_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=False)

for i, data in enumerate(training_loader):
    spectrogram, labels, audio_path = data
    features = model.forward(
        spectrogram,
        output_attentions=False,
        return_dict=True,
    )
    pooled_features = features.pooler_output
    